Streamlined Version to Run Process for New Datasets

In [11]:
import numpy as np
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord

In [12]:
# Data Variable ----------------------------------------------------------------
data_filename = "Resources/notable_data.csv"

dtype = [('Name', 'U50'),('RA (J2000)', 'U12'),('Dec (J2000)', 'U12')]
data = np.genfromtxt(data_filename, delimiter=',',dtype=dtype,names=True,encoding='utf-8')

ra = data['RA_J2000']
dec = data['Dec_J2000']

sky_coords = SkyCoord(ra=ra, dec=dec, unit=(u.hourangle, u.deg), frame='icrs')
gal_coords = sky_coords.galactic

In [13]:
# Load Fermi LAT catalog (Gamma-ray) -------------------------------------------
fermi_hdul = fits.open('Resources/gll_psc_v35.fit')
fermi_data = fermi_hdul[1].data
fermi_l = np.array([float(row[4]) for row in fermi_data]) * u.deg
fermi_b = np.array([float(row[5]) for row in fermi_data]) * u.deg
fermi_coords = SkyCoord(l=fermi_l, b=fermi_b, frame='galactic')

In [14]:
# Load Chandra catalog (X-ray) -------------------------------------------------
dtype_chandra = [
    ('blank', 'f8'),('name', 'U20'),('ra', 'U15'),('dec', 'U15'),
    ('lii', 'f8'),('bii', 'f8'),('error_ellipse_r0', 'f8'),
    ('error_ellipse_r1', 'f8'),('error_ellipse_angle', 'f8')
]
chandra_data = np.genfromtxt('Resources/chandra.txt', delimiter='|', skip_header=3, dtype=dtype_chandra, invalid_raise=False)
chandra_coords = SkyCoord(l=chandra_data['lii'] * u.deg, b=chandra_data['bii'] * u.deg, frame='galactic')

In [20]:
# Load Swift catalog (X-ray) ---------------------------------------------------
dtypeee = [('blank', 'f8'),('source_number', 'i8'), ('name', 'U25'), ('ra', 'U15'), ('dec', 'U15'), ('lii', 'f8'), ('bii', 'f8'), ('error_radius', 'f8'), ('count_rate_fb', 'f8'), ('count_rate_fb_pos_err', 'f8'), ('count_rate_fb_neg_err', 'f8'), ('pow_flux_a', 'f8'), ('pow_flux_a_pos_err', 'f8'), ('pow_flux_a_neg_err', 'f8'), ('apec_flux_a', 'f8'), ('apec_flux_a_pos_err', 'f8'), ('apec_flux_a_neg_err', 'f8')]
data5 = np.genfromtxt('Resources/swift.txt', delimiter='|', skip_header=3, dtype=dtypeee, invalid_raise=False)

go5 = data5['lii'] * u.deg
ga5 = data5['bii'] * u.deg

swift_coords = SkyCoord(l=go5,b=ga5, frame='galactic')

In [21]:
# Matching function ------------------------------------------------------------
def match_catalog(target_coords, catalog_coords):
    idx, sep2d, _ = target_coords.match_to_catalog_sky(catalog_coords)
    return idx, sep2d

fermi_idx, fermi_sep = match_catalog(gal_coords, fermi_coords)
chandra_idx, chandra_sep = match_catalog(gal_coords, chandra_coords)
swift_idx, swift_sep = match_catalog(gal_coords, swift_coords)

In [22]:
# Output -----------------------------------------------------------------------
for i in range(len(gal_coords)):
    print(f"\nObject {i+1}")
    print(f"Galactic: l = {gal_coords.l[i].value:.3f}, b = {gal_coords.b[i].value:.3f}")
    print(f"Fermi Closest Index: {fermi_idx[i]}, Sep = {fermi_sep[i].arcmin:.2f} arcmin")
    print(f"Chandra Closest Name: {chandra_data['name'][chandra_idx[i]]}, Sep = {chandra_sep[i].arcmin:.2f} arcmin")
    print(f"Swift Closest Src: {swift_data['src'][swift_idx[i]]}, Sep = {swift_sep[i].arcmin:.2f} arcmin")


Object 1
Galactic: l = 60.836, b = -3.696
Fermi Closest Index: 6197, Sep = 74.87 arcmin
Chandra Closest Name: 2CXO J195943.8+22412, Sep = 2.56 arcmin
Swift Closest Src: , Sep = 4.86 arcmin

Object 2
Galactic: l = 63.170, b = 13.978
Fermi Closest Index: 5743, Sep = 65.50 arcmin
Chandra Closest Name: 2CXO J185334.4+33015, Sep = 0.16 arcmin
Swift Closest Src: , Sep = 2.22 arcmin

Object 3
Galactic: l = 130.939, b = -10.503
Fermi Closest Index: 441, Sep = 26.43 arcmin
Chandra Closest Name: 2CXO J014220.8+51350, Sep = 0.64 arcmin
Swift Closest Src: , Sep = 33.20 arcmin

Object 4
Galactic: l = 96.499, b = 30.946
Fermi Closest Index: 5298, Sep = 88.16 arcmin
Chandra Closest Name: 2CXO J175113.1+66402, Sep = 16.02 arcmin
Swift Closest Src: , Sep = 25.37 arcmin

Object 5
Galactic: l = 321.910, b = -4.746
Fermi Closest Index: 4237, Sep = 24.17 arcmin
Chandra Closest Name: 2CXO J154220.8-59130, Sep = 120.59 arcmin
Swift Closest Src: , Sep = 39.36 arcmin

Object 6
Galactic: l = 19.571, b = 0.685
